In [1]:
from scholarly import scholarly

search_query = scholarly.search_author('Jens Meiler')
#author = next(search_query, None)
#scholarly.fill(author)

# not working because google blocked me

In [2]:
from tqdm import tqdm
import requests

jens_orcid = '0000-0001-8945-193X'

orcid_api_url = f"https://pub.orcid.org/v3.0/{jens_orcid}/works"
headers = {"Accept": "application/json"}

response = requests.get(orcid_api_url, headers=headers)

In [3]:
data = response.json()
works = data.get('group', [])

auto_dois = []

for work in works:
        work_summary = work.get('work-summary', [])
        for summary in work_summary:
            if 'external-ids' in summary:
                external_ids = summary['external-ids']['external-id']
                for external_id in external_ids:
                    if external_id.get('external-id-type') == 'doi':
                        auto_dois.append('https://doi.org/' + external_id.get('external-id-value'))

print(len(auto_dois), 'of', len(works))

267 of 298


In [4]:
manual_dois = """https://doi.org/10.1101/2024.08.06.606224 

https://doi.org/10.48550/arXiv.2411.09820 

https://doi.org/10.1101/2024.06.20.599843 

https://doi.org/10.26434/chemrxiv-2024-hr692 

https://doi.org/10.1021/acschemneuro.4c00086 

https://doi.org/10.1021/acssynbio.3c00753 

https://doi.org/10.1371/journal.pcbi.1011939 

https://doi.org/10.1021/acs.jctc.3c01081 

https://doi.org/10.1101/2022.11.15.516482  

https://doi.org/10.1016/j.sbi.2023.102645

https://doi.org/10.1609/aaai.v37i12.26679 

https://doi.org/10.1016/j.bpj.2022.11.011 

https://doi.org/10.1101/2023.04.17.537185 

https://doi.org/10.3389/fmolb.2023.1121962 

https://doi.org/10.1016/j.str.2022.11.012 

https://doi.org/10.1073/pnas.2206129119 

https://doi.org/10.1158/0008-5472.CAN-21-3798 

https://doi.org/10.7554/eLife.75751

https://doi.org/10.3389/fphar.2022.833099 

https://doi.org/10.1021/acs.jproteome.1c00410 

https://doi.org/10.1021/acs.jcim.0c01001 

https://doi.org/10.1021/acs.jcim.0c01001 

https://doi.org/10.1109/3DV50981.2020.00103

https://doi.org/10.48550/arXiv.2007.07029 

https://doi.org/10.1007/s10822-019-00199-8 

https://doi.org/10.3390/molecules22091576 

https://doi.org/10.1016/j.bmcl.2016.07.071

https://doi.org/10.1007/s10822-016-9895-2 

https://doi.org/10.1007/s10822-015-9893-9 

https://doi.org/10.1124/pr.112.007336 

https://doi.org/10.3390/molecules18010735 

https://doi.org/10.3390/molecules17089971 

https://doi.org/10.1109/CIBCB.2012.6217248

https://doi.org/10.1109/CIBCB.2012.6217246

https://doi.org/10.1109/CIBCB.2011.5948478

https://doi.org/10.1002/cmdc.201100510 

https://doi.org/10.1109/CIBCB.2011.5948478

https://doi.org/10.1016/j.bpj.2010.12.2344 

https://doi.org/10.1016/j.bmcl.2010.07.061

https://doi.org/10.1109/CIBCB.2009.4925736

https://doi.org/10.1109/CIBCB.2009.4925709

https://doi.org/10.1002/qsar.200330837 

https://doi.org/10.1023/A:1023060720156 

https://doi.org/10.1006/jmre.2002.2599

https://doi.org/10.1021/ci0102970 

https://doi.org/10.1021/ci000021c

https://doi.org/10.1007/PL00010287"""

manual_dois = manual_dois.split()

In [25]:
len(set(manual_dois))

45

In [6]:
joint_dois = set(manual_dois)
joint_dois.update(auto_dois)
joint_dois = list(joint_dois)
intersect_dois = list(set(manual_dois) & set(auto_dois))
print(len(joint_dois), len(intersect_dois))

291 16


In [7]:
from doiref import CrossRefClient

session = CrossRefClient()

ref = session.doi2json('http://dx.doi.org/10.1109/3DV50981.2020.00103')
ref

{'indexed': {'date-parts': [[2024, 10, 29]],
  'date-time': '2024-10-29T09:11:08Z',
  'timestamp': 1730193068708,
  'version': '3.28.0'},
 'reference-count': 28,
 'publisher': 'IEEE',
 'license': [{'start': {'date-parts': [[2020, 11, 1]],
    'date-time': '2020-11-01T00:00:00Z',
    'timestamp': 1604188800000},
   'content-version': 'vor',
   'delay-in-days': 0,
   'URL': 'https://ieeexplore.ieee.org/Xplorehelp/downloads/license-information/IEEE.html'},
  {'start': {'date-parts': [[2020, 11, 1]],
    'date-time': '2020-11-01T00:00:00Z',
    'timestamp': 1604188800000},
   'content-version': 'stm-asf',
   'delay-in-days': 0,
   'URL': 'https://doi.org/10.15223/policy-029'},
  {'start': {'date-parts': [[2020, 11, 1]],
    'date-time': '2020-11-01T00:00:00Z',
    'timestamp': 1604188800000},
   'content-version': 'stm-asf',
   'delay-in-days': 0,
   'URL': 'https://doi.org/10.15223/policy-037'}],
 'content-domain': {'domain': [], 'crossmark-restriction': False},
 'published-print': {'date

In [8]:
corresponding = ['Meiler']

In [9]:
def formatSPP(ref):
    s = ""
    preprint = False
    for author in ref['author']:
        s += author['given'][0] + ". " + author['family'] + ","
        if author['family'] in corresponding:
            s += '*'
        s += ' '
    s = s[:-1] + '\n'
    s += ref['title'] + '\n'
    if 'container-title-short' in ref:
        s += ref['container-title-short'] + " "
    elif 'container-title' in ref and ref['container-title']:
        s += ref['container-title'] + " "
    else:
        if ref['publisher'] == 'Cold Spring Harbor Laboratory':
            s += 'bioRxiv '
            preprint = True
        elif ref['publisher'] == 'American Chemical Society (ACS)':
            s += 'ChemRxiv '
            preprint = True
        elif ref['publisher'] == 'arXiv':
            s += 'arXiv '
            preprint = True
        else:
            raise KeyError('container-title-short')
    if 'created' in ref:
        s += str(ref['created']['date-parts'][0][0])
    elif 'issued' in ref:
        s += str(ref['issued']['date-parts'][0][0])
    else:
        raise KeyError('year')
    if not preprint:
        if 'volume' in ref : s += ", " + str(ref['volume'])
        if 'page' in ref : s += ", " + str(ref['page'])
    s += ". " + ref['URL']
    return s, preprint


In [17]:
all_refs = {}

for doi in tqdm(joint_dois):
    all_refs[doi] = session.doi2json(doi)

100%|██████████| 291/291 [03:23<00:00,  1.43it/s]


In [55]:
ai_dois = set()

buzzwords = [' AI ', ' ML ', 'intelligence', 'learning', 'training', 'alphafold', 'qsar', 'neural network', 
             'esm',
             'computational methods in drug discovery', 'ligand-based virtual screen for the discovery of novel m5 inhibitor chemotypes',
             'computer-aided design and biological evaluation of diazaspirocyclic']

for doi in joint_dois:
    ref = all_refs[doi]
    title = str(ref.get('title', "")).casefold()
    abstract = str(ref.get('abstract', "")).casefold()
    buzzed = False
    for bw in buzzwords:
        if bw.casefold() in title or bw.casefold() in abstract:
            ai_dois.add(doi)
            buzzed = True
            #print(title, doi, bw)
            break
    if not buzzed:
        continue
        print(doi)
        print(title)
        print(abstract)

print(len(ai_dois))


68


In [56]:
errors = []

for doi in list(ai_dois):
    ref = all_refs[doi]
    try:
        #if 'created' in ref:
        #    if ref['created']['date-parts'][0][0] < 2022:
        #        continue
        #elif 'issued' in ref:
        #    if ref['issued']['date-parts'][0][0] < 2022:
        #        continue
        citation, preprint = formatSPP(ref)
        #if not preprint:
        #    print(citation, '\n')
        print(citation, '\n')
    except KeyError as e:
        errors.append([e.args[0], ref['URL'], ref['publisher']])
print(errors)

S. Jiang, Z. Su, N. Bloodworth, Y. Liu, C. Martina, D. Harrison, J. Meiler,*
Machine learning application to predict binding affinity between peptide containing non-canonical amino acids and HLA0201
bioRxiv 2024. http://dx.doi.org/10.1101/2024.11.19.624425 

G. Sliwoski, S. Kothiwale, J. Meiler,* E. Lowe,
Computational Methods in Drug Discovery
Pharmacol Rev 2014, 66, 334-395. http://dx.doi.org/10.1124/pr.112.007336 

J. Leman,, R. Mueller, M. Karakas, N. Woetzel, J. Meiler,*
Simultaneous prediction of protein secondary structure and transmembrane spans
Proteins 2013, 81, 1127-1140. http://dx.doi.org/10.1002/prot.24258 

G. Sliwoski, J. Mendenhall, J. Meiler,*
Autocorrelation descriptor improvements for QSAR: 2DA_Sign and 3DA_Sign
J Comput Aided Mol Des 2015, 30, 209-217. http://dx.doi.org/10.1007/s10822-015-9893-9 

Y. Liu, Y. Wang, O. Vu, R. Moretti, B. Bodenheimer, J. Meiler,* T. Derr,
Interpretable Chirality-Aware Graph Neural Network for Quantitative Structure Activity Relationshi